# GraphSAGE Implementation Test
> Getting familiar with DGL and GraphSAGE

#### Imports

In [1]:
import argparse
import time
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
from dgl.nn.pytorch.conv import SAGEConv
import matplotlib.pyplot as plt

Using backend: pytorch


#### Random

In [2]:
import random
random.seed(1)

#### Disable Warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

#### Dataset

- social network
- nodes being people
- connected if they know each other
- features contain
    - `City` they live in
    - `Street` they live in
    - `Age`
- `School` being the label which the GNN aims to predict

In [4]:
def generate_DGLGraph(node_amount):
    print(" [+] Initialize Graph")
    # initialize Graph
    G = dgl.DGLGraph()
    
    # Add node_amount nodes
    G.add_nodes(node_amount)
    G.ndata['feat'] = torch.zeros((node_amount, 3), dtype=torch.float)
    G.ndata['label'] = torch.zeros(node_amount, dtype=torch.int64)
    
    print(" [+] Generate Features and Labels")
    for i in range(node_amount):
        # print(f' [+] Generate Features and Labels: {i} / {node_amount}', end='\r')
        # features
        street = random.randint(0, 49)
        city   = 0 if street < 10 else int(street / 10)
        age    = random.randint(5, 20)
        G.nodes[i].data['feat'] = torch.tensor([[city, street, age]], dtype=torch.float)
        
        # labels
        school = random.randint(0, 9)
        G.nodes[i].data['label'] = torch.tensor([school], dtype=torch.int64)
    
    # edges
    pairs = []
    src_list, dst_list = [], []
    print(" [+] Generate Edges")
    for i in range(node_amount):
        for j in range(node_amount):
            # print(f' [+] Generate Edges: {i * j} / {node_amount * node_amount}', end='\r')
            # no self loops
            if i == j:
                continue
            
            if know_each_other(G.ndata, i, j) and (i, j) not in pairs:
                src_list.append(i)
                src_list.append(j)
                dst_list.append(j)
                dst_list.append(i)
                pairs.append((i, j))
                pairs.append((j, i))
                
    G.add_edges(src_list, dst_list)
    
    # masks
    print(" [+] Generate Masks")
    G.ndata['train_mask'] = torch.zeros(node_amount, dtype=torch.bool)
    G.ndata['val_mask'] = torch.zeros(node_amount, dtype=torch.bool)
    G.ndata['test_mask'] = torch.zeros(node_amount, dtype=torch.bool)
    
    train_val_split = int(node_amount * 0.2)
    val_test_split  = train_val_split + int(node_amount * 0.4)
        
    for a in range(node_amount):
        G.ndata['train_mask'][a] = a < train_val_split
        G.ndata['val_mask'][a] = a >= train_val_split and a < val_test_split
        G.ndata['test_mask'][a] = a >= val_test_split
   
    # return Graph, number of classes, number of edges / 2
    return G, 10, len(pairs) / 2

The label `School` is also used for deciding whether they know each other or not

In [5]:
def know_each_other(data, i, j, threshold=0.5):
    score = 0.0
    
    x, x_school = data['feat'][i], data['label'][i].item()
    y, y_school = data['feat'][j], data['label'][j].item()
    
    # City : Street : School & Age
    score += 0.25 if x[0].item() == y[0].item() else 0.1
    score += 0.25 if abs(x[1].item() - y[1].item()) < 4 else 0.1
    score += 0.5 if x_school == y_school and abs(x[2].item() - y[2]) < 2 else 0.2 if x_school == y_school else 0.0 

    return True if score >= threshold else False

#### Generate new Social Network

In [6]:
AMOUNT_OF_MEMBERS = 300
social_network_graph, num_classes, num_edges = generate_DGLGraph(AMOUNT_OF_MEMBERS)

 [+] Initialize Graph
 [+] Generate Features and Labels
 [+] Generate Edges
 [+] Generate Masks


#### Visualize Graph

In [7]:
visualize = False

In [8]:
if visualize:
    nx_graph = social_network_graph.to_networkx().to_undirected()
    pos = nx.kamada_kawai_layout(nx_graph)
    nx.draw(nx_graph, pos, node_color=[[.6, .4, .8]])

#### GraphSAGE model

In [9]:
class GraphSAGE(nn.Module):
    
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type))
        # output layer
        self.layers.append(SAGEConv(n_hidden, n_classes, aggregator_type)) # activation None

        
    def forward(self, graph, inputs):
        h = self.dropout(inputs)
        for l, layer in enumerate(self.layers):
            h = layer(graph, h)
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

#### Classification

In [10]:
def evaluate(model, graph, features, labels, nid):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits[nid]
        labels = labels[nid]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [11]:
def main(g, num_classes, num_edges, gpu=0):
    # load and preprocess dataset
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    in_feats = features.shape[1]
    n_classes = num_classes
    n_edges = num_edges
    
    print("""--------Data statistics--------
    
    Edges   %d
    Classes %d
    
    Train samples %d
    Val samples   %d
    Test samples  %d""" %
          (n_edges, n_classes,
           train_mask.int().sum().item(),
           val_mask.int().sum().item(),
           test_mask.int().sum().item()))

    if gpu < 0:
        cuda = False
    else:
        cuda = True
        torch.cuda.set_device(gpu)
        features = features.cuda()
        labels = labels.cuda()
        train_mask = train_mask.cuda()
        val_mask = val_mask.cuda()
        test_mask = test_mask.cuda()
        print("\n    Cuda in use", gpu)

    train_nid = train_mask.nonzero().squeeze()
    val_nid = val_mask.nonzero().squeeze()
    test_nid = test_mask.nonzero().squeeze()

    # graph preprocess and calculate normalization factor
    g = dgl.remove_self_loop(g)
    n_edges = g.number_of_edges()
    if cuda:
        g = g.int().to(gpu)

    # create GraphSAGE model
    model = GraphSAGE(in_feats,         # input features shape
                      16,               # nodes in hidden layers
                      n_classes,        # number of classes
                      2,                # number of hidden layers
                      F.relu,           # activation function
                      0.5,              # drop-out percentage
                      'gcn')            # aggregator type

    if cuda:
        model.cuda()

    # use optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)

    # initialize graph
    dur = []
    print("\n\n--------Training process--------\n")
    for epoch in range(200):
        model.train()
        t0 = time.time()

        # forward
        logits = model(g, features)
        loss = F.cross_entropy(logits[train_nid], labels[train_nid])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        dur.append(time.time() - t0)

        acc = evaluate(model, g, features, labels, val_nid)
        print("Epoch {:03d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
              "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

    print()
    acc = evaluate(model, g, features, labels, test_nid)
    print("Test Accuracy {:.4f}".format(acc))

In [12]:
main(social_network_graph, num_classes, num_edges)

--------Data statistics--------
    
    Edges   6487
    Classes 10
    
    Train samples 60
    Val samples   120
    Test samples  120

    Cuda in use 0


--------Training process--------

Epoch 000 | Time(s) 0.3798 | Loss 21.0778 | Accuracy 0.0833 | ETputs(KTEPS) 34.16
Epoch 001 | Time(s) 0.1920 | Loss 20.5304 | Accuracy 0.0833 | ETputs(KTEPS) 67.58
Epoch 002 | Time(s) 0.1292 | Loss 17.6111 | Accuracy 0.1000 | ETputs(KTEPS) 100.43
Epoch 003 | Time(s) 0.0979 | Loss 10.8145 | Accuracy 0.0750 | ETputs(KTEPS) 132.49
Epoch 004 | Time(s) 0.0790 | Loss 6.7248 | Accuracy 0.0917 | ETputs(KTEPS) 164.18
Epoch 005 | Time(s) 0.0664 | Loss 7.8457 | Accuracy 0.0917 | ETputs(KTEPS) 195.38
Epoch 006 | Time(s) 0.0574 | Loss 8.3588 | Accuracy 0.0917 | ETputs(KTEPS) 226.06
Epoch 007 | Time(s) 0.0506 | Loss 5.7713 | Accuracy 0.1000 | ETputs(KTEPS) 256.27
Epoch 008 | Time(s) 0.0454 | Loss 6.3088 | Accuracy 0.1333 | ETputs(KTEPS) 286.02
Epoch 009 | Time(s) 0.0411 | Loss 4.7238 | Accuracy 0.1167 | ETput

Epoch 131 | Time(s) 0.0059 | Loss 2.1637 | Accuracy 0.1083 | ETputs(KTEPS) 2204.01
Epoch 132 | Time(s) 0.0059 | Loss 2.1263 | Accuracy 0.1083 | ETputs(KTEPS) 2209.82
Epoch 133 | Time(s) 0.0059 | Loss 2.1172 | Accuracy 0.1083 | ETputs(KTEPS) 2213.14
Epoch 134 | Time(s) 0.0058 | Loss 2.1638 | Accuracy 0.1167 | ETputs(KTEPS) 2218.44
Epoch 135 | Time(s) 0.0058 | Loss 2.1390 | Accuracy 0.1083 | ETputs(KTEPS) 2224.87
Epoch 136 | Time(s) 0.0058 | Loss 2.1729 | Accuracy 0.1083 | ETputs(KTEPS) 2229.27
Epoch 137 | Time(s) 0.0058 | Loss 2.2002 | Accuracy 0.1083 | ETputs(KTEPS) 2237.60
Epoch 138 | Time(s) 0.0058 | Loss 2.1787 | Accuracy 0.1000 | ETputs(KTEPS) 2245.59
Epoch 139 | Time(s) 0.0058 | Loss 2.1933 | Accuracy 0.1000 | ETputs(KTEPS) 2254.08
Epoch 140 | Time(s) 0.0057 | Loss 2.1230 | Accuracy 0.0917 | ETputs(KTEPS) 2261.06
Epoch 141 | Time(s) 0.0057 | Loss 2.1480 | Accuracy 0.0833 | ETputs(KTEPS) 2267.62
Epoch 142 | Time(s) 0.0057 | Loss 2.1542 | Accuracy 0.0833 | ETputs(KTEPS) 2276.14
Epoc